In [1]:
# Import some basic libraries and functions for this tutorial.
import numpy as np
import os

from pydrake.common import temp_directory
from pydrake.geometry import StartMeshcat
from pydrake.math import RigidTransform, RollPitchYaw, RotationMatrix
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder
from pydrake.visualization import AddDefaultVisualization, ModelVisualizer

print("Check!")

Check!


In [2]:
from pydrake.all import (
    AddMultibodyPlantSceneGraph, DiagramBuilder, 
    FindResourceOrThrow, GenerateHtml, InverseDynamicsController, 
    MultibodyPlant, Parser, Simulator)
from pydrake.multibody.jupyter_widgets import MakeJointSlidersThatPublishOnCallback
import pydrake
from pydrake import geometry
from pydrake.math import RigidTransform, RollPitchYaw, RotationMatrix 
from pydrake.solvers import MathematicalProgram, Solve
from pydrake.systems.jupyter_widgets import PoseSliders, WidgetSystem
from ipywidgets import ToggleButton, ToggleButtons
from functools import partial
from pydrake.all import (
    JointIndex, PiecewisePolynomial, JacobianWrtVariable,
    eq, ge,  AutoDiffXd, SnoptSolver, IpoptSolver,  
    AddUnitQuaternionConstraintOnPlant, PositionConstraint, OrientationConstraint
    )
print("Check!")

Check!


# 1 Setting up the scene
#### some notes:

from pydrake.all import (
    autoDiffToGradientMatrix, initializeAutoDiffGivenGradientMatrix, autoDiffToValueMatrix, autoDiffToGradientMatrix)

**These imports seem to be renovated, should have equivalent. Do check!** 

In [3]:
h = 1e-3
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=h)
parser = Parser(plant)

In [4]:
dir1 = "/Users/xiao/0_codes/ICBM_drake/models/objects"
dir2 = "/Users/xiao/0_codes/ICBM_drake/models/ycb/sdf/"
table_file = os.path.join(dir1, "table_top.sdf")
panda_file = "/Users/xiao/0_codes/ICBM_drake/models/franka_description/urdf/panda_arm_hand.urdf"
box_file = os.path.join(dir2, "003_cracker_box.sdf")

# scene_dir = ("/Users/xiao/0_codes/ICBM_drake/scenes/scene1")
# These add multiple models and returns a list
table_modelInstance = parser.AddModels(table_file)[0]  # this return pydrake.multibody.tree.ModelInstanceIndex
panda_modelInstance = parser.AddModels(panda_file)[0]
box_modelInstance = parser.AddModels(box_file)[0]

In [10]:
print(table_modelInstance)
print(type(table_modelInstance))

ModelInstanceIndex(2)
<class 'pydrake.multibody.tree.ModelInstanceIndex'>


In [11]:
# get table-top frame in the world, here the dummy is for getting familiar with Drake
table_top_frame = plant.GetFrameByName("table_top_center")
robot_base = plant.GetFrameByName("panda_link0")

# move the table for a bit
X_W_table = RigidTransform(
  R=RotationMatrix([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0],]),
  p=[0.0, 0.0, 0.0]
) # or similar: X_W_table = RigidTransform(RotationMatrix.Identity(), [0., -0.4, 0.])

plant.WeldFrames(plant.world_frame(), table_top_frame, X_W_table) # 2nd arg relative to 1st arg transformed by X_W_table 

# fix panda to table top.
plant.WeldFrames(table_top_frame, robot_base, RigidTransform(RotationMatrix.Identity(), [0., -0.4, 0.]))


# plant.WeldFrames(X_W_table, table_top_frame)
print(f"table_top_frame is {table_top_frame}")

""" These will be depracated on 2023-12-01
Deprecated:
    Use parser.AddModels() instead. To port the 2-argument form,
    rename models using parser.SetAutoRenaming() and
    plant.RenameModelInstance(). See PR #19978 for more details. This
    will be removed from Drake on or after 2023-12-01.
  parser.AddModelFromFile(table_file, "table") """
# # using add model from file 
# parser.AddModelFromFile(table_file, "table")
# parser.AddModelFromFile(box_file, "box")

"""
# This is explicitly fixing the table to world frame. What happens if it is not
# explicitly fixed here but "not publishing to update its state" ?? See later sections.  

# fix the centre of table to world
table_body_frame = plant.GetFrameByName("table_top_center")
plant.WeldFrames(plant.world_frame(), table_body_frame) 
"""

# finalize the plant
plant.Finalize()
plant.set_name("table_with_box")

# now build the diagram
diagram = builder.Build()

table_top_frame is <FixedOffsetFrame name='table_top_center' index=3 model_instance=2>


## 1 On setting fixed body pose in world
Use $plant.WeldFrames(parentFrame,\;childFrame, \;Transformation)$ to fix objects in the scene. For fixed objects, this needs to be done before $plant.Finalize()$
### 1.1 for getting frames:
plant.world_frame(), 
plant.GetFrameByName()
### 1.2 using drake transformations:
The transformation X should be of type "*pydrake.math.RigidTransform*"
e.g.:
X_W_table = RigidTransform(
  R=RotationMatrix([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0],]),
  p=[0.0, 0.0, 0.0],)

In [12]:
# get free body box and its id in the plant
box_body = plant.GetBodyByName("base_link_cracker") # get the body instance
box_body_id = plant.GetBodyFrameIdIfExists(box_body.index()) # get the body's id in the plant

table_top_body = plant.GetBodyByName("table_top_link") # since only one from each pool
table_top_frame_id = plant.GetBodyFrameIdIfExists(table_top_body.index())

panda_base = plant.GetBodyByName("panda_link0")
panda_base_frame_id = plant.GetBodyFrameIdIfExists(panda_base.index())

panda_hand = plant.GetBodyByName("panda_hand")
panda_hand_frame_id = plant.GetBodyFrameIdIfExists(panda_hand.index())

print(f"box_body_frame is {box_body} \n")
print(f"box_body_id is {box_body_id}")

print(type(box_body))

print(type(box_body_id))

# this 
print(box_body.body_frame()) # this get the body frame from the body instance

box_body_frame is <RigidBody name='base_link_cracker' index=14 model_instance=4> 

box_body_id is <FrameId value=268>
<class 'pydrake.multibody.tree.RigidBody'>
<class 'pydrake.geometry.FrameId'>
<BodyFrame name='base_link_cracker' index=27 model_instance=4>


In [13]:
panda_base.body_frame()
panda_hand.body_frame()

<BodyFrame name='panda_hand' index=13 model_instance=3>

In [14]:
# create the default diagram's context 
context = diagram.CreateDefaultContext()

# get the mutable context for updates
mutable_context = plant.GetMyContextFromRoot(context) # seems equivalent to .GetMyMutableContextFromRoot()

# # move the table for a bit
# X_W_table = RigidTransform(RotationMatrix.Identity(), [0., -0.4, 0.])

# plant.SetFreeBodyPose(mutable_context, plant.GetBodyByName("table_top_link"), X_W_table)

# move the box a bit relative to the table-top
X_table_box = RigidTransform(RollPitchYaw(np.asarray([0, 0, 0]) * np.pi / 180), p=[0.1, -0.2, 0.])
X_W_table_dummy = table_top_frame.CalcPoseInWorld(mutable_context)
X_W_box = X_W_table_dummy.multiply(X_table_box)
plant.SetFreeBodyPose(mutable_context, box_body, X_W_box)

In [15]:
meshcat = StartMeshcat()
# not yet publish any context for visualization

INFO:drake:Meshcat listening for connections at http://localhost:7001


In [16]:
# Need this because a==b returns True even if a = AutoDiffXd(1, [1, 2]), b= AutoDiffXd(2, [3, 4])
# That's the behavior of AutoDiffXd in C++, also.
def autoDiffArrayEqual(a,b):
    return np.array_equal(a, b) and np.array_equal(ExtractGradient(a), ExtractGradient(b))

In [17]:
from pydrake.all import (AutoDiffXd, ExtractGradient)
import numpy as np
a = AutoDiffXd(1, [1, 2])
b = AutoDiffXd(2, [3, 4]) 

In [18]:
N = 100

In [19]:
"""
one default context of the diagram per time step and each corresponds to one plant context? 
It this needed?
To use SceneGraph, the plant context need to be derived from diagram
"""

# Create one context per timestep to maximize cache hits
context_list = [diagram.CreateDefaultContext() for i in range(N)] 
plant_context_list = [plant.GetMyContextFromRoot(context_list[i]) for i in range(N)]

#or just:
# plant_context_list = [plant.CreateDefaultContext() for i in range(N)]

In [20]:
# convert to AutoDiff compatible
ad_diagram = diagram.ToAutoDiffXd()
ad_plant = ad_diagram.GetSubsystemByName("table_with_box")

In [21]:
# create new AutoDiff context to maximize cache hits
""" To use SceneGraph, the plant context need to be derived from diagram """
ad_context_list = [ad_diagram.CreateDefaultContext() for i in range(N)]
ad_plant_context_list = [ad_plant.GetMyContextFromRoot(ad_context_list[i]) for i in range(N)]

In [23]:
# the context here is the very first one 
""" Notices here the box is an list of pydrake.multibody.tree.ModelInstanceIndex after parser.AddModels(...) """
box_mass = sum(plant.get_body(index).get_mass(context) for index in plant.GetBodyIndices(box_modelInstance))
panda_mass = sum(plant.get_body(index).get_mass(context) for index in plant.GetBodyIndices(panda_modelInstance))
gravity = plant.gravity_field().gravity_vector()

print(f"The box's mass is: {box_mass}\n")
print(f"Total mass of panda is: {panda_mass}\n")
print(f"The gravity vector is: {gravity}\n")

The box's mass is: 0.411

Total mass of panda is: 18.92

The gravity vector is: [ 0.    0.   -9.81]



# 2 setting up the prog() and bind decVars

In [24]:
# time step constraint
T = 0.5
prog = MathematicalProgram()

# time steps
h_var = prog.NewContinuousVariables(N-1, "h")
print(f"h_var has type: {type(h_var)}, and shape: {h_var.shape}\n")
print(f"h_var[0] has type: {type(h_var[0])}\n")

h_var has type: <class 'numpy.ndarray'>, and shape: (99,)

h_var[0] has type: <class 'pydrake.symbolic.Variable'>



In [25]:
# constraints on time
prog.AddBoundingBoxConstraint([0.5*T/(N-1)]*(N-1), [2.0*T/(N-1)]*(N-1), h_var)
prog.AddLinearConstraint(sum(h_var) >= 0.9*T)
prog.AddLinearConstraint(sum(h_var) <= 1.5*T)
prog.SetInitialGuess(h_var, [T/(N-1)]*(N-1))

In [46]:
# constraints on q, v, vdot
nq_box = plant.num_positions(box_modelInstance)
nv_box = plant.num_velocities(box_modelInstance)

nq_arm = plant.num_positions(panda_modelInstance)
nv_arm = plant.num_velocities(panda_modelInstance)

# 7 + 2, 2 are two 'fixed' finger joints
assert nq_arm == 9, "# of joints does not match, something is wrong!"

# the return type of prog.NewContinuousVariables() is numpy.ndarray
q_box_vars = prog.NewContinuousVariables(nq_box, N, "q_box") # [row, col]: [nq_o, N]
v_box_vars = prog.NewContinuousVariables(nv_box, N, "v_box")
q_arm_vars = prog.NewContinuousVariables(nq_arm-2, N, "q_arm")
v_arm_vars = prog.NewContinuousVariables(nv_arm-2, N, "q_arm")

""" Initialize the contact forces here. 
    These are the lambdas scalars, not unit wrench (directional) that should come from kinematics """
# #Add lambdas into the decVar, the n_oo, n_oe are arbituary here as placeholders
# #n_oo, n_oe should be determined from camera observations and assumptions
# n_ho = 1 # for non-prehensile
# n_oo = 3
# n_oe = 4 
# lambda_ho = prog.NewContinuousVariables(n_ho, N, "f_ho")
# lambda_oo = prog.NewContinuousVariables(n_oo, N, "f_oo")
# lambda_oe = prog.NewContinuousVariables(n_oe, N, "f_oe")

""" This should not be here. 
    constraints on acceleration should come from the dynamics. """
# vdot_vars = prog.NewContinuousVariables(nv_o, N-1, "vdot_o") 

' This should not be here. \n    constraints on acceleration should come from the dynamics. '

In [51]:
# intial constraint
q0_box = plant.GetPositions(mutable_context, box_modelInstance)
v0_box = plant.GetVelocities(mutable_context, box_modelInstance)
q0_arm = plant.GetPositions(mutable_context, panda_modelInstance)[:7] # excluding the fingers
v0_arm = plant.GetVelocities(mutable_context, panda_modelInstance)[:7] 
                             

print(f"The box initial q is: {q0_box} \n")
print(f"The box initial v is: {v0_box} \n")
print(f"The panda initial q is: {q0_arm} \n")
print(f"The panda initial v is: {v0_arm} \n")

The box initial q is: [ 1.   0.   0.   0.   0.1 -0.2  0. ] 

The box initial v is: [0. 0. 0. 0. 0. 0.] 

The panda initial q is: [0. 0. 0. 0. 0. 0. 0.] 

The panda initial v is: [0. 0. 0. 0. 0. 0. 0.] 



In [52]:
# set init v and vdot if needed
vdot0_box = np.array([0., 0., 0., 0., 0., -9.81]) # not sure if this is needed
v0_box = np.array([0., 0., 0., 0., 0., 0.]) # twist [wx, wy, wz, vx, vy, vz]

# set box constraints on the initial states
prog.AddBoundingBoxConstraint(q0_box, q0_box, q_box_vars[:, 0]) # at t = 0
prog.AddBoundingBoxConstraint(v0_box, v0_box, v_box_vars[:, 0]) # at t = 0
prog.AddBoundingBoxConstraint(q0_arm, q0_arm, q_arm_vars[:, 0]) 
prog.AddBoundingBoxConstraint(v0_arm, v0_arm, v_arm_vars[:, 0]) 

In [37]:
""" # this is adds unit quaternion constraints to all quaternion joints in the plant

# add unit quaternion constraint for each time point
for i in range(N):
    # This constraint is added to the generalized positions not just the rotation part?? Double check this in C++
    # yes, to generalized positions
    AddUnitQuaternionConstraintOnPlant(plant, q_o_vars[:, i], prog) # The decision variables for the generalized position of the plant.
"""

# what I need is unit quaternion constraints on the object at all time points
from pydrake.multibody.inverse_kinematics import UnitQuaternionConstraint
for i in range(N):
    prog.AddConstraint(UnitQuaternionConstraint(), q_o_vars[3:, i]) # double check if positions are arranged [x, y, z, w, i, j, k]

In [35]:
tester = np.arange(7)
print(tester)
print(tester[4:])
print(tester[3:])

[0 1 2 3 4 5 6]
[4 5 6]
[3 4 5 6]


In [38]:
q_o_vars.shape

(7, 100)

In [23]:
plant.num_positions() # after fixing the table to world, this drops from 14 to 7

7

In [29]:
floating_bodies = plant.GetFloatingBaseBodies() # this returns a python set
print(f"Total: {len(floating_bodies)} floating bodies in the world")

Total: 1 floating bodies in the world


# 3 Adding constraints on the decVars


In [52]:
n_u = plant.num_actuated_dofs()
B = np.identity(n_u) 

""" n_f: number of contacts """
n_f = n_ho + n_oo + n_oe # 

# the manipulator equation constraint
def manipulator_equation_cst(decVars, context_index):
    """
    this function implements the system dynamics in implicit form, as non-linear eq constraint, using implicit Euler integration: 
    
    0 = [ Buₙ₊₁ + ∑ᵢ (Jᵢ_WBᵀ(qₙ₊₁)ᵀ * Fᵢ_AB_W(λᵢ,ₙ₊₁)) + tau_g(qₙ₊₁) - C(qₙ₊₁, Vₙ₊₁) ] * dt - M(qₙ₊₁) * (Vₙ₊₁ - Vₙ)
    
    decVars: []
    The format of the input to the Eval() function is a vector containing: {vₙ, qₙ₊₁, vₙ₊₁, uₙ₊₁, λₙ₊₁, dt},
    where λₙ₊₁ is a concatenation of lambdas for all contacts
    """
    # unpack decVars
    v_n, q_next, v_next, u_next, lambdas, dt = np.split(decVars, [nq_o, 
                                                                  nq_o+nq_v, 
                                                                  nq_o+nq_v*2, 
                                                                  nq_o+nq_v*2+n_u, 
                                                                  nq_o+nq_v*2+n_u+n_f])
    
    # update context position and velocity
    if isinstance(decVars[0], AutoDiffXd):
        if not autoDiffArrayEqual(q_next, ad_plant.GetPositions(ad_plant_context_list[context_index], box)):
            ad_plant.SetPositions(ad_plant_context_list[context_index], box, q_next)
        if not autoDiffArrayEqual(v_next, ad_plant.GetVelocities(ad_plant_context_list[context_index], box)):
            ad_plant.SetVelocities(ad_plant_context_list[context_index], box, v_next)
        
        # this is needed to commodate ad and non-ad plant, so no need to code it twice
        eval_plant = ad_plant
        eval_plant_context = ad_plant_context_list[context_index]
            
    else:
        if not np.array_equal(q, plant.GetPositions(plant_context_list[context_index], box)):
            plant.SetPositions(plant_context_list[context_index], box, q_next)
        if not np.array_equal(v, plant.GetVelocities(plant_context_list[context_index], cracker_box)):
            plant.SetVelocities(plant_context_list[context_index], box, v_next)

        eval_plant = plant
        eval_plant_context = plant_context_list[context_index]
    
    # control with selection matrix
    y = B @ u_next
    g_term = eval_plant.CalcGravityGeneralizedForces(eval_plant_context)

    # gravity
    y += g_term
    c_term = eval_plant.CalcBiasTerm(eval_plant_context)

    # velocity product (or bias term) 
    y -= c_term
    M = eval_plant.CalcMassMatrixViaInverseDynamics(eval_plant_context)

    # now the contact, which has two part: contact jacobian and the contact wrench
    # first off, the contact Jacobian 
    # I think the one is needed here for the manipulation equation cst.

    """ Jv_V_WCa reads the spatial velocity(V) Jacobian of contact point a (Ca) expressed in {W} w.r.t the generalize v 
        the point in body frame is required by SELECTION """
    Jv_V_WCa = eval_plant.CalcJacobianSpatialVelocity(eval_plant_context, 
                                                      JacobianWrtVariable.kV, 
                                                      frameB, 
                                                      p_BCb, 
                                                      eval_plant.world_frame(), 
                                                      eval_plant.world_frame())
    """
    or
    Jv_v_WCa = eval_plant.CalcJacobianTranslationalVelocity()
    Jv_v_WCb = eval_plant.CalcJacobianTranslationalVelocity()
    
    these are needed to compute the relative velocity at contact point.

    Make sure I understand all the jacobians tools in drake!
    """
    
    # initialize the ee_term which is the same dim as the arm dof
    ee_wrench = np.zeros(nv_o)
    
    # need to compute the contact Jacobian based on selected points (or project onto surface) 
    # rather than using SceneGraph inspector
    y += 

AttributeError: module 'numpy' has no attribute 'identify'

In [41]:
t = np.arange(10)
print(t)

a = np.split(t, [2, 4, 6])
print(a)

[0 1 2 3 4 5 6 7 8 9]
[array([0, 1]), array([2, 3]), array([4, 5]), array([6, 7, 8, 9])]


In [42]:
plant.num_actuated_dofs()

0

In [50]:
isinstance(q_o_vars[0, 0], AutoDiffXd)

False

In [51]:
type(q_o_vars[0][0])

pydrake.symbolic.Variable

In [53]:
np.zeros((5))

array([0., 0., 0., 0., 0.])

In [54]:
np.zeros(5)

array([0., 0., 0., 0., 0.])